In [68]:
import vk_api
import json
from tqdm import tqdm
import pandas as pd

In [4]:
vk_session = vk_api.VkApi(token='')

In [5]:
vk = vk_session.get_api()
tools = vk_api.VkTools(vk_session)

In [28]:
# качаем стену
wall = tools.get_all('wall.get', 100, {'owner_id': -108367462, 'extended': 1})
wall = wall['items']

In [36]:
wall[1]

{'attachments': [{'photo': {'access_key': 'af96c322dc119596bd',
    'album_id': -7,
    'date': 1481363739,
    'height': 656,
    'id': 456239093,
    'owner_id': -108367462,
    'photo_1280': 'https://pp.vk.me/c604417/v604417028/39b3a/LmSodUAh7ro.jpg',
    'photo_130': 'https://pp.vk.me/c604417/v604417028/39b37/ouRD5_2-050.jpg',
    'photo_604': 'https://pp.vk.me/c604417/v604417028/39b38/VEqoMdhSbA8.jpg',
    'photo_75': 'https://pp.vk.me/c604417/v604417028/39b36/qnSr2Jzy4Wo.jpg',
    'photo_807': 'https://pp.vk.me/c604417/v604417028/39b39/oQvzgZ_tahU.jpg',
    'post_id': 728,
    'text': '#куспеху@talentedme #прокрастинация #цитата\n\nЗдесь вы найдете много мотивирующих цитат https://goo.gl/TDC0aU',
    'user_id': 100,
    'width': 1000},
   'type': 'photo'}],
 'comments': {'can_post': 1, 'count': 0},
 'date': 1481363739,
 'from_id': -108367462,
 'id': 728,
 'likes': {'can_like': 1, 'can_publish': 1, 'count': 1, 'user_likes': 0},
 'owner_id': -108367462,
 'post_source': {'type': 'ap

In [58]:
# likes
for post in tqdm(wall):
    likes = vk.likes.getList(filter='likes', item_id=post['id'], owner_id=post['owner_id'], count=1000, **{'type': 'post'})
    likes = likes['items']
    post['likes']['users'] = likes 
    
    reposts = vk.likes.getList(filter='copies', item_id=post['id'], owner_id=post['owner_id'], count=1000, **{'type': 'post'})
    reposts = reposts['items']
    post['reposts']['users'] = reposts

100%|██████████| 449/449 [06:18<00:00,  1.18it/s]


In [62]:
# comments
for post in tqdm(wall):
    comments = vk.wall.getComments(post_id=post['id'], owner_id=post['owner_id'], need_likes=1, count=1000)
    comments = comments['items']
    post['comments']['items'] = comments

100%|██████████| 449/449 [03:10<00:00,  2.44it/s]


In [65]:
wall[2]

{'attachments': [{'photo': {'access_key': '08456483528097f1fe',
    'album_id': -7,
    'date': 1481356051,
    'height': 656,
    'id': 456239092,
    'owner_id': -108367462,
    'photo_1280': 'https://pp.vk.me/c837625/v837625028/e28f/MVd6chqO-jk.jpg',
    'photo_130': 'https://pp.vk.me/c837625/v837625028/e28c/vNXi7JNyH5U.jpg',
    'photo_604': 'https://pp.vk.me/c837625/v837625028/e28d/BNRKmJZ1MCE.jpg',
    'photo_75': 'https://pp.vk.me/c837625/v837625028/e28b/zf7vyYu5ndk.jpg',
    'photo_807': 'https://pp.vk.me/c837625/v837625028/e28e/vZ8ih7C_yXg.jpg',
    'post_id': 725,
    'text': '[10 УРОКОВ ДЛЯ ВАШЕЙ КАРЬЕРЫ ]\nРаспечатайте и\xa0повесьте на\xa0стену!\n\nЛегко смотреть на\xa0людей, у\xa0которых карьера идёт в\xa0гору соразмерно с\xa0зарплатой, и\xa0завистливо вздыхать. Сложнее остановиться и\xa0задуматься о\xa0том, что же пошло не\xa0так, и\xa0суметь вовремя это\xa0исправить. \n\n1. Жизнь коротка\n\nКак бы банально это\xa0ни\xa0звучало, но\xa0у\xa0вас всегда есть выбор. И\xa0свою

In [66]:
wall[2].keys()

dict_keys(['post_type', 'post_source', 'comments', 'owner_id', 'reposts', 'id', 'from_id', 'date', 'text', 'likes', 'attachments'])

In [63]:
with open('data/vk/wall.json', mode='w', encoding='utf-8') as f:
    json.dump(wall, f, ensure_ascii=False, indent=2)

In [116]:
# какие-то таблицы

In [112]:
posts_predf = []

for post in tqdm(wall):
    row = (post['id'], post['date'], post['likes']['count'], post['reposts']['count'], post['comments']['count'], post['text'])
    posts_predf.append(row)
    
posts_df = pd.DataFrame(posts_predf)

100%|██████████| 449/449 [00:00<00:00, 226869.35it/s]


In [113]:
posts_df.columns = ['post_id', 'date', 'likes', 'reposts', 'comments', 'text']
posts_df.head()

,post_id,date,likes,reposts,comments,text
0,698,1480407159,12,7,3,10 видеокурсов по основам менеджмента для руко...
1,728,1481363739,1,0,0,#куспеху@talentedme #прокрастинация #цитата\n\...
2,725,1481356051,11,4,2,[10 УРОКОВ ДЛЯ ВАШЕЙ КАРЬЕРЫ ]\nРаспечатайте и...
3,724,1481263024,6,3,0,[КАК ВЫЙТИ ИЗ ТВОРЧЕСКОГО КРИЗИСА?]\nСоветы от...
4,722,1481204686,6,1,0,Здесь вы найдете много мотивирующих цитат http...


In [114]:
posts_df.to_csv('data/vk/wall.csv')

In [108]:
comments_predf = []

for post in tqdm(wall):
    for comment in post['comments']['items']:
        row = (post['id'], comment['id'], comment['from_id'], comment['date'], comment['likes']['count'], comment['text'])
        comments_predf.append(row)
    
comments_df = pd.DataFrame(comments_predf)
comments_df.columns = ['post_id', 'comment_id', 'user_id', 'date', 'likes', 'text']
comments_df.to_csv('data/vk/wall-comments.csv')

100%|██████████| 449/449 [00:00<00:00, 149868.10it/s]


In [109]:
comments_df.head()

,post_id,comment_id,user_id,date,likes,text
0,698,699,375376551,1480408965,1,"Так и задумано, что все ссылки битые?"
1,698,700,336340028,1480414409,0,"[id375376551|Juli], спасибо за информацию, на ..."
2,698,701,375376551,1480416213,0,"[id336340028|Сергей], спасибо, теперь вроде от..."
3,725,726,173458547,1481357005,1,"Будьте проще, если что то не получается, жизнь..."
4,725,727,173458547,1481357246,0,"Записывайте свои цели и подробный план, что бы..."


In [110]:
likes_predf = []

for post in tqdm(wall):
    for user in post['likes']['users']:
        row = (post['id'], user, 0)
        likes_predf.append(row)
    for user in post['reposts']['users']:
        row = (post['id'], user, 1)
        likes_predf.append(row)
    
likes_df = pd.DataFrame(likes_predf)
likes_df.columns = ['post_id', 'user_id', 'is_repost']
likes_df.to_csv('data/vk/wall-likes-and-reposts.csv')

100%|██████████| 449/449 [00:00<00:00, 116703.38it/s]


In [111]:
likes_df.head()

,post_id,user_id,is_repost
0,698,119871102,0
1,698,190355493,0
2,698,145759099,0
3,698,242956477,0
4,698,98213622,0


In [115]:
# нужные таблицы

In [118]:
len(likes_df.user_id.unique())

1340

In [ ]:
liked_group_themes_predf = []

for user in likes_df.user_id.unique():
    # TODO